In [ ]:
import numpy as np
import pandas as pd
import json
import requests
import re
import os
from datetime import datetime as dt
from pandas.io.json import json_normalize
import geocoder
import matplotlib.pylab as plt
import seaborn as sns
import shapefile
from sklearn import preprocessing
import time

%matplotlib inline

df_lat_long = pd.read_csv("df_lat_long.csv")
df_lat_long = df_lat_long[:100]

date_t = dt.today().strftime("%Y%m%d")

#https://foursquare.com/developers/apps 申请网站
ID = 'I2E3FGJX0Z12Y0PHJ22W3WZBSUNDYWPJAQCFMSHJQBSAQJKG' 
Paw = 'K13KOCRW22OVDQJBMYNBD5B4EMWBPBLU5CCDX4UWVL3V3T03'

def getNearbyPlaces(names, latitudes, longitudes, date_t, radius = 500, limit = 100,):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            ID, Paw, date_t, lat, lng, radius, limit)
        
        number_food = requests.get(url+'&section=food').json()['response']['totalResults']
        number_drink = requests.get(url+'&section=drinks').json()['response']['totalResults'] + requests.get(url+'&section=coffee').json()['response']['totalResults']
        
        number_shop = requests.get(url+'&section=shops').json()['response']['totalResults']
        number_art = requests.get(url+'&section=arts').json()['response']['totalResults']
        number_trending = requests.get(url+'&section=trending').json()['response']['totalResults']
        
        url_price_low = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&price=1'.format(
            ID, Paw, date_t, lat, lng, radius, limit)
        price_low = requests.get(url_price_low).json()['response']['totalResults']
        
        url_price_high = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&price=3,4'.format(
            ID, Paw, date_t, lat, lng, radius, limit)
        price_high = requests.get(url_price_high).json()['response']['totalResults']
        
        venues_list.append([(name, lat, lng, 
            number_food,number_drink,number_shop,number_art,number_trending,price_low,price_high)])
                            
                            
    selected_ven = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    selected_ven.columns = ['street_name', 'Latitude', 'Longitude', 'Food', 'Drink', 'Shope', 'Art', 'Trending', 'Price_low', 'Price_high']
    return(selected_ven)

df_venues = getNearbyPlaces(names = df_lat_long['street_name'],
                            latitudes = df_lat_long['Latitude'],
                            longitudes = df_lat_long['Longitude'],
                            date_t = dt.today().strftime("%Y%m%d"))

In [1]:
import numpy as np
import pandas as pd
import json
import requests
import re
import os
from datetime import datetime as dt
from pandas.io.json import json_normalize
import geocoder
import matplotlib.pylab as plt
import seaborn as sns
import shapefile
from sklearn import preprocessing
%matplotlib inline

df_lat_long = pd.read_csv("df_lat_long.csv")
df_lat_long = df_lat_long[:100]

date_t = dt.today().strftime("%Y%m%d")

#https://foursquare.com/developers/apps 申请网站
ID = 'I2E3FGJX0Z12Y0PHJ22W3WZBSUNDYWPJAQCFMSHJQBSAQJKG' 
Paw = 'K13KOCRW22OVDQJBMYNBD5B4EMWBPBLU5CCDX4UWVL3V3T03'

venues_list=[]
names = df_lat_long['street_name']
latitudes = df_lat_long['Latitude']
longitudes = df_lat_long['Longitude']
date_t = dt.today().strftime("%Y%m%d")
radius = 500
limit = 100
    
for name, lat, lng in zip(names, latitudes, longitudes):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        ID, Paw, date_t, lat, lng, radius, limit)

    number_food = requests.get(url+'&section=food').json()['response']['totalResults']
    number_drink = requests.get(url+'&section=drinks').json()['response']['totalResults'] + requests.get(url+'&section=coffee').json()['response']['totalResults']

    number_shop = requests.get(url+'&section=shops').json()['response']['totalResults']
    number_art = requests.get(url+'&section=arts').json()['response']['totalResults']
    number_trending = requests.get(url+'&section=trending').json()['response']['totalResults']

    url_price_low = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&price=1'.format(
        ID, Paw, date_t, lat, lng, radius, limit)
    price_low = requests.get(url_price_low).json()['response']['totalResults']

    url_price_high = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&price=3,4'.format(
        ID, Paw, date_t, lat, lng, radius, limit)
    price_high = requests.get(url_price_high).json()['response']['totalResults']

    venues_list.append([(name, lat, lng, 
        number_food,number_drink,number_shop,number_art,number_trending,price_low,price_high)])


selected_ven = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
selected_ven.columns = ['street_name', 'Latitude', 'Longitude', 'Food', 'Drink', 'Shope', 'Art', 'Trending', 'Price_low', 'Price_high']

In [3]:
selected_ven.to_csv('selected_ven1.csv')

In [6]:
selected_ven1 = pd.read_csv('selected_ven1.csv')
selected_ven1

,Unnamed: 0,street_name,Latitude,Longitude,Food,Drink,Shope,Art,Trending,Price_low,Price_high
0,0,ANG MO KIO AVE 10,1.367602,103.856844,23,18,23,5,17,8,2
1,1,ANG MO KIO AVE 4,1.382319,103.838706,10,5,13,6,11,5,0
2,2,ANG MO KIO AVE 5,1.376825,103.847556,12,13,18,4,10,1,1
3,3,ANG MO KIO AVE 1,1.364355,103.843642,15,12,9,4,17,6,0
4,4,ANG MO KIO AVE 3,1.369691,103.840906,19,12,11,7,15,6,1
...,...,...,...,...,...,...,...,...,...,...,...
95,95,WATERLOO ST,1.299426,103.852085,79,102,136,62,87,26,5
96,96,NEW MKT RD,1.287705,103.845216,130,131,65,18,126,35,16
97,97,TANJONG PAGAR PLAZA,1.275570,103.842993,88,102,65,10,104,33,11
98,98,QUEEN ST,1.302628,103.856076,118,115,113,24,106,38,8


In [12]:

df_lat_long = pd.read_csv("df_lat_long.csv")
df_lat_long = df_lat_long[100:200]

names = df_lat_long['street_name']
latitudes = df_lat_long['Latitude']
longitudes = df_lat_long['Longitude']

    
for name, lat, lng in zip(names, latitudes, longitudes):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        ID, Paw, date_t, lat, lng, radius, limit)

    number_food = requests.get(url+'&section=food').json()['response']['totalResults']
    number_drink = requests.get(url+'&section=drinks').json()['response']['totalResults'] + requests.get(url+'&section=coffee').json()['response']['totalResults']

    number_shop = requests.get(url+'&section=shops').json()['response']['totalResults']
    number_art = requests.get(url+'&section=arts').json()['response']['totalResults']
    number_trending = requests.get(url+'&section=trending').json()['response']['totalResults']

    url_price_low = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&price=1'.format(
        ID, Paw, date_t, lat, lng, radius, limit)
    price_low = requests.get(url_price_low).json()['response']['totalResults']

    url_price_high = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&price=3,4'.format(
        ID, Paw, date_t, lat, lng, radius, limit)
    price_high = requests.get(url_price_high).json()['response']['totalResults']

    venues_list.append([(name, lat, lng, 
        number_food,number_drink,number_shop,number_art,number_trending,price_low,price_high)])


selected_ven2 = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
selected_ven2.columns = ['street_name', 'Latitude', 'Longitude', 'Food', 'Drink', 'Shope', 'Art', 'Trending', 'Price_low', 'Price_high']
selected_ven2.to_csv('selected_ven2.csv')

In [13]:
df_lat_long = pd.read_csv("df_lat_long.csv")
df_lat_long = df_lat_long[200:300]

names = df_lat_long['street_name']
latitudes = df_lat_long['Latitude']
longitudes = df_lat_long['Longitude']

    
for name, lat, lng in zip(names, latitudes, longitudes):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        ID, Paw, date_t, lat, lng, radius, limit)

    number_food = requests.get(url+'&section=food').json()['response']['totalResults']
    number_drink = requests.get(url+'&section=drinks').json()['response']['totalResults'] + requests.get(url+'&section=coffee').json()['response']['totalResults']

    number_shop = requests.get(url+'&section=shops').json()['response']['totalResults']
    number_art = requests.get(url+'&section=arts').json()['response']['totalResults']
    number_trending = requests.get(url+'&section=trending').json()['response']['totalResults']

    url_price_low = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&price=1'.format(
        ID, Paw, date_t, lat, lng, radius, limit)
    price_low = requests.get(url_price_low).json()['response']['totalResults']

    url_price_high = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&price=3,4'.format(
        ID, Paw, date_t, lat, lng, radius, limit)
    price_high = requests.get(url_price_high).json()['response']['totalResults']

    venues_list.append([(name, lat, lng, 
        number_food,number_drink,number_shop,number_art,number_trending,price_low,price_high)])


selected_ven3 = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
selected_ven3.columns = ['street_name', 'Latitude', 'Longitude', 'Food', 'Drink', 'Shope', 'Art', 'Trending', 'Price_low', 'Price_high']
selected_ven3.to_csv('selected_ven3.csv')

KeyError: 'totalResults'

In [ ]:
time.sleep(4000)
df_lat_long = pd.read_csv("df_lat_long.csv")
df_lat_long = df_lat_long[300:400]

names = df_lat_long['street_name']
latitudes = df_lat_long['Latitude']
longitudes = df_lat_long['Longitude']

    
for name, lat, lng in zip(names, latitudes, longitudes):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        ID, Paw, date_t, lat, lng, radius, limit)

    number_food = requests.get(url+'&section=food').json()['response']['totalResults']
    number_drink = requests.get(url+'&section=drinks').json()['response']['totalResults'] + requests.get(url+'&section=coffee').json()['response']['totalResults']

    number_shop = requests.get(url+'&section=shops').json()['response']['totalResults']
    number_art = requests.get(url+'&section=arts').json()['response']['totalResults']
    number_trending = requests.get(url+'&section=trending').json()['response']['totalResults']

    url_price_low = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&price=1'.format(
        ID, Paw, date_t, lat, lng, radius, limit)
    price_low = requests.get(url_price_low).json()['response']['totalResults']

    url_price_high = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&price=3,4'.format(
        ID, Paw, date_t, lat, lng, radius, limit)
    price_high = requests.get(url_price_high).json()['response']['totalResults']

    venues_list.append([(name, lat, lng, 
        number_food,number_drink,number_shop,number_art,number_trending,price_low,price_high)])


selected_ven4 = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
selected_ven4.columns = ['street_name', 'Latitude', 'Longitude', 'Food', 'Drink', 'Shope', 'Art', 'Trending', 'Price_low', 'Price_high']
selected_ven4.to_csv('selected_ven4.csv')

In [ ]:
time.sleep(4000)
df_lat_long = pd.read_csv("df_lat_long.csv")
df_lat_long = df_lat_long[400:500]

names = df_lat_long['street_name']
latitudes = df_lat_long['Latitude']
longitudes = df_lat_long['Longitude']

    
for name, lat, lng in zip(names, latitudes, longitudes):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        ID, Paw, date_t, lat, lng, radius, limit)

    number_food = requests.get(url+'&section=food').json()['response']['totalResults']
    number_drink = requests.get(url+'&section=drinks').json()['response']['totalResults'] + requests.get(url+'&section=coffee').json()['response']['totalResults']

    number_shop = requests.get(url+'&section=shops').json()['response']['totalResults']
    number_art = requests.get(url+'&section=arts').json()['response']['totalResults']
    number_trending = requests.get(url+'&section=trending').json()['response']['totalResults']

    url_price_low = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&price=1'.format(
        ID, Paw, date_t, lat, lng, radius, limit)
    price_low = requests.get(url_price_low).json()['response']['totalResults']

    url_price_high = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&price=3,4'.format(
        ID, Paw, date_t, lat, lng, radius, limit)
    price_high = requests.get(url_price_high).json()['response']['totalResults']

    venues_list.append([(name, lat, lng, 
        number_food,number_drink,number_shop,number_art,number_trending,price_low,price_high)])


selected_ven5 = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
selected_ven5.columns = ['street_name', 'Latitude', 'Longitude', 'Food', 'Drink', 'Shope', 'Art', 'Trending', 'Price_low', 'Price_high']
selected_ven5.to_csv('selected_ven5.csv')

In [ ]:
time.sleep(4000)
df_lat_long = pd.read_csv("df_lat_long.csv")
df_lat_long = df_lat_long[500:]

names = df_lat_long['street_name']
latitudes = df_lat_long['Latitude']
longitudes = df_lat_long['Longitude']

    
for name, lat, lng in zip(names, latitudes, longitudes):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        ID, Paw, date_t, lat, lng, radius, limit)

    number_food = requests.get(url+'&section=food').json()['response']['totalResults']
    number_drink = requests.get(url+'&section=drinks').json()['response']['totalResults'] + requests.get(url+'&section=coffee').json()['response']['totalResults']

    number_shop = requests.get(url+'&section=shops').json()['response']['totalResults']
    number_art = requests.get(url+'&section=arts').json()['response']['totalResults']
    number_trending = requests.get(url+'&section=trending').json()['response']['totalResults']

    url_price_low = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&price=1'.format(
        ID, Paw, date_t, lat, lng, radius, limit)
    price_low = requests.get(url_price_low).json()['response']['totalResults']

    url_price_high = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&price=3,4'.format(
        ID, Paw, date_t, lat, lng, radius, limit)
    price_high = requests.get(url_price_high).json()['response']['totalResults']

    venues_list.append([(name, lat, lng, 
        number_food,number_drink,number_shop,number_art,number_trending,price_low,price_high)])


selected_ven6 = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
selected_ven6.columns = ['street_name', 'Latitude', 'Longitude', 'Food', 'Drink', 'Shope', 'Art', 'Trending', 'Price_low', 'Price_high']
selected_ven6.to_csv('selected_ven6.csv')